# File for fetching github issues for Milestone 2

In [3]:
# import requests
# import json
# import pandas as pd

# # GitHub API Configuration
# TOKEN = ""  # Add your GitHub Personal Access Token here (optional but recommended)
# headers = {"Accept": "application/vnd.github.v3+json"}
# if TOKEN:
#     headers["Authorization"] = f"token {TOKEN}"

# # Define repository to fetch issues from
# repo = "microsoft/vscode"

# # Define parameters to filter issues (fetch only open issues with "type:bug" label)
# params = {"state": "open", "labels": "Type: Bug", "per_page": 100}  # Max per_page is 100

# # List to store issues data
# issues_list = []
# page = 1  # Start from page 1
# issue_counter = 0

# while True:
#     params["page"] = page
#     api_url = f"https://api.github.com/repos/{repo}/issues"
#     response = requests.get(api_url, headers=headers, params=params)

#     if response.status_code == 200:
#         issues = response.json()

#         if not issues:  # If no more issues are returned, stop fetching
#             break

#         print(f"Fetched {len(issues)} issues from page {page}")
        
#         # Extract relevant fields
#         for issue in issues:
#             issue_counter += 1
#             issues_list.append({
#                 "issue_number": issue_counter,
#                 "repository": repo,
#                 "title": issue.get("title"),
#                 "description": issue.get("body", "No description provided"),
#                 "labels": [label["name"] for label in issue.get("labels", [])],
#                 "created_at": issue.get("created_at"),
#                 "comments": issue.get("comments"),
#                 "reactions": issue.get("reactions", {}).get("total_count", 0),
#                 "url": issue.get("html_url")
#             })

#         page += 1  # Move to the next page

#     else:
#         print(f"Failed to fetch data from {repo}: {response.status_code}, {response.text}")
#         break  # Stop if an error occurs

# # Save the data to CSV and JSON
# df = pd.DataFrame(issues_list)
# df.to_csv("react_all.csv", index=False)
# df.to_json("react_all.json", orient="records")

# print(f"Total issues fetched: {len(issues_list)}")
# print("Issues data saved successfully!")

import requests
import json
import pandas as pd

# GitHub API Configuration
TOKEN = ""  # Add your GitHub Personal Access Token here (optional but recommended)
headers = {"Accept": "application/vnd.github.v3+json"}
if TOKEN:
    headers["Authorization"] = f"token {TOKEN}"

# Define repository to fetch issues from
repo = "microsoft/vscode"

# Define parameters to filter issues (fetch only open issues with "type:bug" label)
params = {"state": "open", "labels": "bug", "per_page": 100}  # Max per_page is 100

# List to store issues data
issues_list = []
page = 1  # Start from page 1
issue_counter = 0
MAX_ISSUES = 500  # Limit the number of issues

while issue_counter < MAX_ISSUES:
    params["page"] = page
    api_url = f"https://api.github.com/repos/{repo}/issues"
    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        issues = response.json()

        if not issues:  # If no more issues are returned, stop fetching
            break

        print(f"Fetched {len(issues)} issues from page {page}")
        
        # Extract relevant fields
        for issue in issues:
            if issue_counter >= MAX_ISSUES:
                break  # Stop fetching when limit is reached
            
            issue_counter += 1
            issues_list.append({
                "issue_number": issue_counter,
                "repository": repo,
                "title": issue.get("title"),
                "description": issue.get("body", "No description provided"),
                "labels": [label["name"] for label in issue.get("labels", [])],
                "created_at": issue.get("created_at"),
                "comments": issue.get("comments"),
                "reactions": issue.get("reactions", {}).get("total_count", 0),
                "url": issue.get("html_url")
            })

        page += 1  # Move to the next page

    else:
        print(f"Failed to fetch data from {repo}: {response.status_code}, {response.text}")
        break  # Stop if an error occurs

# Save the data to CSV and JSON
df = pd.DataFrame(issues_list)
df.to_csv("vscode_issues_500.csv", index=False)
df.to_json("vscode_issues_500.json", orient="records")

print(f"Total issues fetched: {len(issues_list)}")
print("Issues data saved successfully!")



Fetched 100 issues from page 1
Fetched 100 issues from page 2
Fetched 100 issues from page 3
Fetched 100 issues from page 4
Fetched 100 issues from page 5
Total issues fetched: 500
Issues data saved successfully!
